## GRIB: modifying field values

In [1]:
import earthkit.data as ekd

ds = ekd.from_source("file", "test4.grib")
f_ori = ds[0]
f_ori.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,500,20070101,1200,0,an,0,regular_ll


#### Modifying metadata

When we modify the metadata with :py:meth:`~data.core.fieldlist.Field.clone` a new field is created.

In [2]:
f_new = f_ori.clone(level=700)
f_new.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,700,20070101,1200,0,an,0,regular_ll


In [3]:
f_new = f_ori.clone(level="abc", my_key="123")
f_new.ls(extra_keys="my_key")

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType,my_key
0,ecmf,t,isobaricInhPa,abc,20070101,1200,0,an,0,regular_ll,123


In [4]:
def _f(field, key, original_metadata):
    return original_metadata["param"] + str(original_metadata["level"])

f_new = f_ori.clone(custom_name=_f)
f_new.ls(extra_keys="custom_name")

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType,custom_name
0,ecmf,t,isobaricInhPa,500,20070101,1200,0,an,0,regular_ll,t500


#### Modifying values

In [5]:
f_new = f_ori.clone(values=f_ori.values + 1)
f_new.values[0:3], f_ori.values[0:3]

(array([229.04600525, 229.04600525, 229.04600525]),
 array([228.04600525, 228.04600525, 228.04600525]))

#### Modify both metadata and values

In [6]:
f_new = f_ori.clone(values=f_ori.values + 1, level=700)
f_new.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,700,20070101,1200,0,an,0,regular_ll


In [7]:
f_new.values[0:3]

array([229.04600525, 229.04600525, 229.04600525])

#### Saving the modifield field into a GRIB file

In [8]:
f_new.save("_modified_field.grib")
ds_new = ekd.from_source("file", "_modified_field.grib")
ds_new[0].ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,700,20070101,1200,0,an,0,regular_ll


In [9]:
ds_new[0].values[0:3]

array([229.04600525, 229.04600525, 229.04600525])

#### Saving is not always possible

Saving the modified field into a GRIB is only possible if the modified metadata is GRIB compatible. 

In the following example with add a custom metadata key called "_level".

In [10]:
f_new = f_ori.clone(_level=700)
f_new.metadata("_level", "level")

(700, 500)

Writing to GRIB is not possible because "_level" is not a valid GRIB key and ecCodes raises an exception.

In [11]:
try:
    f_new.save("_modified_field1.grib")
except Exception as e:
    print(e)

Error setting _level=700
Key/value not found
Traceback (most recent call last):
  File "/Users/cgr/git/earthkit-data/src/earthkit/data/utils/message.py", line 273, in set
    return eccodes.codes_set(self._handle, name, value)
  File "/Users/cgr/git/eccodes-python/gribapi/gribapi.py", line 2140, in grib_set
    grib_set_long(msgid, key, value)
  File "/Users/cgr/git/eccodes-python/gribapi/gribapi.py", line 1006, in grib_set_long
    GRIB_CHECK(lib.grib_set_long(h, key.encode(ENC), value))
  File "/Users/cgr/git/eccodes-python/gribapi/gribapi.py", line 226, in GRIB_CHECK
    errors.raise_grib_error(errid)
  File "/Users/cgr/git/eccodes-python/gribapi/errors.py", line 381, in raise_grib_error
    raise ERROR_MAP[errid](errid)
gribapi.errors.KeyValueNotFoundError: Key/value not found


Key/value not found
